In [ ]:
#Perspective transform 
def perp_transform(img, M):
    warped = cv2.warpPerspective(img, M, dsize = (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    return warped

#Use output of cell above
#Call img for ease of coding
# img=combined
# imshape = img.shape
# img_size = (img.shape[1], img.shape[0])
#print(img_size) Result is image size (1280,720)
#Try to determine src points manually
w,h = 1280,720

src = np.float32([[(200, 720), (570, 470), (720, 470), (1130, 720)]])
dest = np.float32([[(350, 720), (350, 0), (980, 0), (980, 720)]])


M = cv2.getPerspectiveTransform(src, dest)
M_inv = cv2.getPerspectiveTransform(dest, src)

#ran the tracker class below. May need to move that up

warped = perp_transform(dsti, M)

    
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
ax1.imshow(dsti)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(warped)
ax2.set_title('Transformed', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig(input_folder+r"\output_images\transformed.jpg")





In [ ]:
img_t = cv2.undistort(warped, mtx, dist, None, mtx)
grad_x = abs_sobel_thresh(img_t, orient='x', thresh_min=20, thresh_max=130)
grad_y = abs_sobel_thresh(img_t, orient='y', thresh_min=20, thresh_max=130)
hls_binary = hls_select(img_t, thresh = (80,255)) #hls color select
mag_binary = mag_thresh(img_t, sobel_kernel=kernel_size, mag_thresh=(30, 130))
dir_binary = dir_threshold(img_t, sobel_kernel=kernel_size, thresh=(0.7, 1.3))
warped = np.zeros_like(dir_binary)
warped[((grad_x == 1) & (hls_binary == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
ax1.imshow(img_t)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(warped, cmap='gray')
ax2.set_title('Transformed', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig(input_folder+r"\output_images\transformed_2.jpg")


In [ ]:
def pre_process(img):    
    input_folder = r"H:\Courses\Self-Driving Car ND\CarND-P4"
    dist_pickle = pickle.load(open(input_folder+"\camera_cal\calibration_pickle.p","rb"))
    mtx=dist_pickle["mtx"]
    dist=dist_pickle["dist"]
    #undistort the image
    img = undistort_image(img, mtx, dist) #these paramters were calculated above in first cell
    w,h = 1280,720
    src = np.float32([[(200, 720), (570, 470), (720, 470), (1130, 720)]])
    dest = np.float32([[(350, 720), (350, 0), (980, 0), (980, 720)]])
   
    M = cv2.getPerspectiveTransform(src, dest)
    M_inv = cv2.getPerspectiveTransform(dest, src)
    #ran the tracker class below. May need to move that up
    warped = perp_transform(img, M)
    img_t = cv2.undistort(warped, mtx, dist, None, mtx)
    grad_x = abs_sobel_thresh(img_t, orient='x', thresh_min=20, thresh_max=130)
    grad_y = abs_sobel_thresh(img_t, orient='y', thresh_min=20, thresh_max=130)
    hls_binary = hls_select(img_t, thresh = (80,255)) #hls color select
    mag_binary = mag_thresh(img_t, sobel_kernel=kernel_size, mag_thresh=(30, 130))
    dir_binary = dir_threshold(img_t, sobel_kernel=kernel_size, thresh=(0.7, 1.3))
    warped = np.zeros_like(dir_binary)
    warped[((grad_x == 1) & (hls_binary == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    return warped


In [ ]:
# link https://www.youtube.com/watch?v=vWY8YUayf9Q&list=PLAwxTw4SYaPkz3HerxrHlu1Seq8ZA7-5P&index=4
# From the walk through video, link above.      
class tracker():
    
    #when starting a new instance be syre to specify all unassigned variables
    def __init__(self, Mywindow_width, Mywindow_height, Mymargin, My_ym=1, My_xm=1, Mysmooth_factor = 15):
        #list that stores all past (left, right) center set values used to smooth output
        self.recent_centers=[]
        
        #the window pixel width of the center values used to count pixels 
        self.window_width = Mywindow_width
        self.window_height = Mywindow_height
        #pixel distance in both directions to slide
        self.margin = Mymargin
        
        self.ym_per_pix = My_ym  #meters per pixel in vertical axis
        self.xm_per_pix = My_xm  #meters per pixel in horizontal axis
        self.smooth_factor = Mysmooth_factor
        
        
    # the main tracking function for finding and storing lane segment positions
  
    def find_window_centroids(self, warped):

        window_width = self.window_width
        window_height = self.window_height
        margin = self.margin
        
        window_centroids = [] # store the (left, right) window centroid positions per level
        window = np.ones(window_width)  # Create our window template that we will use for convolutions

        # first find the two starting positions for the left and right lane by using to get the vertical image slide
        # and then np.convolve the vertical image slide with the window template
        
        # do the summation for the left side of the bottom quarter of the image
        l_sum = np.sum(warped[int(3 * warped.shape[0] / 4):, :int(warped.shape[1] / 2)], axis=0)
        l_center = np.argmax(np.convolve(window, l_sum)) - window_width/2
        # do the summation for the right side of the bottom quarter of the image
        r_sum = np.sum(warped[int(3 * warped.shape[0] / 4):, int(warped.shape[1] / 2):], axis=0)
        r_center = np.argmax(np.convolve(window, r_sum)) - window_width / 2 + int(warped.shape[1] / 2)

        # Add what we found for the first layer
        window_centroids.append((l_center, r_center))

        # Go through each layer looking for max pixel locations
        for level in range(1, (int)(warped.shape[0]/window_height)):
            # convolve the image into the vertical slice of the image
            image_layer = np.sum(warped[int(warped.shape[0] - (level + 1) * window_height):int(warped.shape[0] - level * window_height), :],
                               axis=0)
            conv_signal = np.convolve(window, image_layer)

            # find the best left centroid by using past left center as a reference
            # use width/2 as offset because convolution signal reference is at right side of window, not center ofwindow
            # look only to the left and right side of the previous maximum

            offset = window_width / 2
            l_min_index = int(max(l_center + offset - margin, 0))
            l_max_index = int(min(l_center + offset + margin, warped.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index]) + l_min_index - offset

            # Find the best right centroid by using past right center as a reference
            r_min_index = int(max(r_center + offset - margin, 0))
            r_max_index = int(min(r_center + offset + margin, warped.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index]) + r_min_index - offset

            # Add what we found for that layer
            window_centroids.append((l_center, r_center))

        self.recent_centers.append(window_centroids)

        # return average values of the line centers, helps to keep the markers from jumping around too much
        # let's look at past 15 values (default for smooth factor)
        return np.average(self.recent_centers[-self.smooth_factor:], axis=0)

In [ ]:
def window_mask(width, height, img_ref, center, level):
    output=np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height), max(0,int(center-width)):min(int(center+width),img_ref.shape[1])]=1
    return output

def process_image(in_img):
    warped = pre_process(in_img)
    
    window_width = 25
    window_height = 80

    curve_centers = tracker(Mywindow_width = window_width, Mywindow_height = window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor = 15)

    window_centroids=curve_centers.find_window_centroids(warped)

    # Points used to draw the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    # points used to find left and right lanes
    rightx=[]
    leftx=[]

    # Step through each level and draw the windows
    for level in range(0,len(window_centroids)):
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        # Window_mask is a function to draw window areas
        l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0],level)
        r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found
        #leftx.append(window_centroids[level][0])
        l_points[(l_points == 255)| ((l_mask==1))] = 255
        r_points[(r_points == 255)| ((r_mask==1))] = 255
    
    #Draw the results
    template=np.array(r_points+l_points,np.uint8) #add left and right window pixels together
    zero_channel = np.zeros_like(template) #create a zero color channel
    template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8)
    warpage=np.array(cv2.merge((warped, warped, warped)),np.uint8) #making original image 3 color channels 
    result=cv2.addWeighted(warpage,1,template,0.5,0.0) #overlay the original road image with window results
    #plt.imshow(result)
    #plt.show()
    yvals=[]
    res_yvals = [] 
    #Starting to look at fitting the polynomials
    yvals=range(0,warped.shape[0])

    res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)

    left_fit=np.polyfit(res_yvals,leftx,2)
    left_fitx=left_fit[0]*yvals*yvals + left_fit[1]*yvals+left_fit[2]
    left_fitx=np.array(left_fitx,np.int32)

    right_fit=np.polyfit(res_yvals,rightx,2)
    right_fitx=right_fit[0]*yvals*yvals + right_fit[1]*yvals+right_fit[2]
    right_fitx=np.array(right_fitx,np.int32)

    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2), axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2), axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    inner_lane = np.array(list(zip(np.concatenate((left_fitx+window_width/2,right_fitx[::-1]-window_width/2), axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)

    road = np.zeros_like(in_img)
    road_bkg=np.zeros_like(in_img)
    cv2.fillPoly(road,[left_lane],color=[255, 0, 0])
    cv2.fillPoly(road,[right_lane],color=[0, 0, 255])
    cv2.fillPoly(road,[inner_lane], color = [0,255,0])
    cv2.fillPoly(road_bkg,[left_lane],color=[255, 255, 255])
    cv2.fillPoly(road_bkg,[right_lane],color=[255, 255, 255])

    road_warped = perp_transform(road,M_inv)
    road_warped_bkg = perp_transform(road_bkg,M_inv)

    base = cv2.addWeighted(in_img,1.0,road_warped_bkg,-1.0,0.0) 
    result = cv2.addWeighted(in_img,1.0,road_warped,0.7,0.0) 

    ym_per_pix = curve_centers.ym_per_pix #meters per pixel in y dimension
    xm_per_pix = curve_centers.xm_per_pix #meters per pixel in x dimension

    curve_fit_cr = np.polyfit(np.array(res_yvals,np.float32)*ym_per_pix, np.array(leftx,np.float32)*xm_per_pix, 2)
    curverad = ((1 + (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])

    # calculate the effect of the car on the road
    camera_center = (left_fitx[-1]+right_fitx[-1])/2
    center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pos = 'right'

    #draw the text showing curvature, offset and speed

    cv2.putText(result,'Radius of Curvature = '+str(round(curverad,3))+'(m)',(50,50) , cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2)
    cv2.putText(result,'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100) , cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2)

    return result


In [ ]:
import os

for pic in os.listdir("H:/Courses/Self-Driving Car ND/CarND-P4/test_images/"):
    img = mpimg.imread("H:/Courses/Self-Driving Car ND/CarND-P4/test_images/" + pic)
    step1=process_image(img)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(step1)
    ax2.set_title('Transformed', fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    mpimg.imsave("H:/Courses/Self-Driving Car ND/CarND-P4/test_images_output/" + pic, step1)
    

In [ ]:
import imageio

from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import pickle
import glob

# dist_pickle = pickle.load(open(input_folder+"\camera_cal\calibration_pickle.p","rb"))
# mtx=dist_pickle["mtx"]
# dist=dist_pickle["dist"]

Input_video = 'H:/Courses/Self-Driving Car ND/CarND-P4/project_video.mp4'
Output_video = 'H:/Courses/Self-Driving Car ND/CarND-P4/project_output.mp4'
print(Input_video)
counter=0
clip1 = VideoFileClip(Input_video)
video_clip = clip1.fl_image(process_image)
%time video_clip.write_videofile(Output_video, audio=False)
print(counter)